In [126]:
import pandas as pd
import numpy as np
import os
import pickle
from fuzzywuzzy import fuzz
from loguru import logger


In [50]:
hsn_code_data = pd.read_csv('hsn_code_des.csv')
statista_data = os.listdir('statista_data_csv')

with open('mappings/amazon_category_list.pickle', 'rb') as handle:
    amazon_category_list = pickle.load(handle)

In [45]:
# amazon_category_list = {
#     'Beauty & Personal Care':{
#     'Makeup':['Body Makeup', 'Eye Makeup', 'Face Makeup', 'Lip Makeup', 'Makeup Palettes', 'Makeup Remover', 'Makeup Sets'],
#     'Skin Care':['Body','Eyes','Face','Lip Care','Maternity','Sets & Kits','Sunscreens & Tanning Products'],
#     'Hair Care':['Hair Styling Accessories','Hair Coloring Products','Hair Cutting Tools','Hair Extensions, Wigs & Accessories',
#     'Hair Loss','Hair Treatment Masks','Hair Perms, Relaxers & Texturizers','Hair Treatment Oils','Scalp Treatments',
#     'Shampoo & Conditioner','Hair Styling Products','Styling Tools & Appliances'],
#     'Perfumes & Fragrances':['Children\'s Fragrance','Fragrance Dusting Powders','Men\'s Fragrances','Fragrance Sets','Women\'s Fragrances'],
#     'Foot, Hand & Nail Care':['Foot & Hand Care','Nail Polish & Decoration Products','Nail Care Products','Hand, Foot & Nail Tools'],
#     'Beauty Tools & Accessories':['Makeup Bags & Cases','Bathing Accessories','Cotton Balls & Swabs','Makeup Brushes & Tools',
#     'Personal Mirrors','Refillable Cosmetic Containers','Shaving & Hair Removal Products','Skin Care Tools'],
#     'Shaving & Hair Removal':['Men\'s Shaving & Hair Removal Products','Women\'s Shaving & Hair Removal Products'],
#     'Personal Care':['Bath & Bathing Accessories','Deodorants & Antiperspirants','Lip Care Products','Oral Care Products',
#     'Piercing & Tattoo Supplies','Body Scrubs & Treatments','Shaving & Hair Removal Products','Skin Care Products'],
#     'Salon & Spa Equipment':['Hair Drying Hoods','Galvanic Facial Machines','Handheld Mirrors','High Frequency Facial Machines',
#     'Manicure Tables','Professional Massage Equipment','Salon & Spa Stools','Spa Beds & Tables','Salon & Spa Chairs',
#     'Spa Storage Systems','Spa Hot Towel Warmers']
#     },
#     'Grocery & Gourmet Food':{
#         'Alcoholic Beverages':['Beer','Cocktail Mixers','Liquor & Spirits','Wine'],
#         'Baby Foods':['Baby Beverages','Cereal & Porridge for Babies','Baby & Toddler Formula','Baby Food Meals','Baby Snack Foods'],
#         'Beverages':['Beverage Syrups & Concentrates','Bottled Beverages, Water & Drink Mixes','Bubble Tea Products','Coffee',
#         'Coffee Gifts','Coffee Substitutes','Hot Chocolate & Malted Drinks','Infusion Mixers for Alcohol & Spirits','Tea Beverages'],
#         'Breads & Bakery':['Bakery & Dessert Gifts','Fresh Bakery Breadcrumbs & Stuffing','Breads','Fresh Bakery Breadsticks','Cakes',
#         'Grocery Cookies','Bakery Desserts','Breakfast Bakery','Pastry Shells & Crusts','Pizza Crusts','Refrigerated Doughs'],
#         'Breakfast Foods':['Applesauce & Fruit Cups','Breakfast & Cereal Bars','Breakfast Syrups & Toppings','Breakfast Cereals','Instant Breakfast Drinks',
#         'Breakfast Pancake, Waffle & Baking Mixes','Toaster Pastries'],
#         'Dairy, Eggs & Plant-Based Alternatives':['Butter, Margarine & Plant-Based Alternatives','Cheeses','Cheese & Charcuterie Gifts','Cheese Assortments & Samplers',
#         'Refrigerated Cheese Dips & Spreads','Cottage Cheeses','Cream Cheeses','Dairy & Plant-Based Yogurt','Eggs & Egg Substitutes','Milks & Creams','Plant-Based Cheese',
#         'Plant-Based Coffee Creamers','Plant-Based Milk','Refrigerated Snack Packs & Lunches','Sour Creams','Whipped Toppings'],
#         'Deli & Prepared Foods':['Fresh Prepared Appetizers','Fresh Prepared Box Lunches','Fresh Prepared Breakfast Foods','Fresh Prepared Casseroles, Pot Pies & Quiches',
#         'Deli Meats & Cheeses','Fresh Prepared Deli Salads','Fresh Prepared Dips, Salsas & Spreads','Fresh Prepared Entrees','Fresh Prepared Pasta & Sauces',
#         'Fresh Prepared Risotto','Fresh Meal Ingredient Kits','Antipasto','Fresh Prepared Sandwiches & Wraps','Fresh Prepared Side Dishes','Fresh Prepared Soups, Stews & Chili'],
#         'Food & Beverage Gifts':['Food & Drink Advent Calendars','Food Assortments & Variety Gifts','Bakery & Dessert Gifts',
#         'Candy & Chocolate Gifts','Cheese & Charcuterie Gifts','Coffee Gifts','Fruit & Nut Gifts','Herb, Spice & Seasoning Gifts',
#         'Jam, Jelly & Sweet Spread Gifts','Meat & Seafood Gifts','Sauce, Gravy & Marinade Gifts','Snack Food Gifts','Gourmet Tea Gifts'],
#         'Fresh Flowers & Live Indoor Plants':['Fresh Flower & Plant Centerpieces','Fresh Cut Flowers','Fresh Flower & Plant Sprays','Fresh Flower & Plant Wreaths','Live Indoor House Plants'],
#         'Fresh Meal Ingredient Kits':[],
#         'Frozen Foods':['Frozen Appetizers & Snacks','Frozen Bread & Dough','Frozen Breakfast Foods','Frozen Desserts & Toppings','Frozen Fruits',
#         'Ice Creams & Frozen Novelties','Frozen Juices','Frozen Meals & Entrees','Frozen Meat','Frozen Pasta & Sauces','Frozen Pizzas','Frozen Potatoes & Onion Rings',
#         'Frozen Seafood','Frozen Vegetables'],
#         'Home Brewing & Winemaking':['Beer Brewing Ingredients','Beer Brewing Recipe Kits''Winemaking Ingredients'],
#         'Meat & Seafood':['Bacon','Beef','Chicken','Foie Gras & Pâtés','Frozen Meat','Hot Dogs & Franks','Lamb Meat','Meat & Seafood Gifts','Pork Meat',
#         'Sausages','Seafood','Turducken','Turkey','Veal Meats','Wild Game & Fowl Meat'],
#         'Meat Substitutes':['Meatless Bacon','Meatless Burgers & Patties','Meatless Deli Slices''Meatless Hot Dogs, Links & Sausages','Meatless Jerky',
#         'Meatless Loaves & Roasts','Meatless Meatballs, Chunks & Crumbles','Meatless Nuggets & Wings','Meatless Pulled & Flaked Meats','Fresh Seitan',
#         'Meatless Strips & Cutlets','Fresh Tempeh','Fresh Tofu'],
#         'Pantry Staples':['Canned, Jarred & Packaged Foods','Condiments & Salad Dressings','Cooking & Baking Grocery Supplies','Dried Beans, Lentils & Peas',
#         'Dried Grains & Rice','Herbs, Spices & Seasonings','Jams, Jellies & Sweet Spreads','Nut & Seed Butters','Olives, Pickles & Relishes','Pasta & Noodles',
#         'Sauces, Gravies & Marinades','Soups, Stocks & Broths'],
#         'Fresh Produce':['Dried Fruits & Vegetables','Fresh Fruits','Fresh Herbs','Fresh Vegetables','Fruit & Nut Gifts','Cooking & Baking Nuts & Seeds',
#         'Fresh Packaged Produce','Fresh Vegetarian Proteins'],
#         'Snacks & Sweets':['Candy & Chocolate','Chocolate','Snack Foods']
#     }
# }

# with open('amazon_category_list.pickle', 'wb') as handle:
#     pickle.dump(amazon_category_list, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# with open('mappings/amazon_category_list.pickle', 'wb') as handle:
#     pickle.dump(amazon_category_list, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [47]:
# with open('mappings/pet_supplies.json') as jsonfile:
#     jsondict = json.load(jsonfile)

# amazon_category_list['Pet Supplies'] = jsondict['Pet Supplies']

In [106]:
df = pd.read_csv('statista_data_csv/'+statista_data[0]+'/'+os.listdir('statista_data_csv/'+statista_data[0])[0])

In [111]:
category_mappings = pd.DataFrame(df['Market'].unique(),columns=['statista_categories_l1'])

In [113]:
for key in amazon_category_list.keys():
    category_mappings['fuzzywuzzy_score_'+key] = category_mappings['statista_categories_l1'].apply(lambda x:fuzz.ratio(key,x))

In [114]:
category_mappings

,statista_categories_l1,fuzzywuzzy_score_Beauty & Personal Care,fuzzywuzzy_score_Grocery & Gourmet Food,fuzzywuzzy_score_Home & Kitchen,fuzzywuzzy_score_Toys & Games,fuzzywuzzy_score_Pet Supplies
0,Consumer Electronics,24,14,35,31,25
1,Household Appliances,19,14,41,38,50
2,"Beauty, Health, Personal & Household Care",67,29,25,23,23
3,Beauty Care,67,30,24,35,35
4,Health Care,48,12,32,26,35
5,Household Care,44,22,36,38,23
6,"Toys, Hobby & DIY",21,26,32,48,7
7,Toys & Baby,30,30,32,70,9
8,Sports & Outdoor,26,42,33,36,21
9,"DIY, Garden & Pets",35,40,31,33,27


In [137]:
country = 'Germany'
hsn_code1 = '1517'
category = 'Butter'
path = 'ecommercedb_data/'+country.title()
try:
    filename1 = [x for x in os.listdir(path) if hsn_code1 in x][0]
    logger.info('filename 1 is {}'.format(filename1))
    files = os.listdir(path+ '/'+filename1)
    logger.info('files are {}'.format(files))
    file_vals = []
    logger.info('fuzzy matching with {}'.format(category))
    for file in files:
        file_vals.append(fuzz.ratio(category,file))
    logger.info('fuzzy matching with {}'.format(file_vals))
    filename2 = files[np.argmax(file_vals)]
    logger.info('filename 2 is {}'.format(filename2))
    path = path+'/'+filename1+'/'+filename2
    req_file = os.listdir(path)[0]
    logger.info('required file path is {}'.format(path+'/'+req_file))
    df = pd.read_excel(path+'/'+req_file)
    df.fillna('NULL',inplace = True)
except:
    path = 'ecommercedb_data/'+country.title()+'/All-categories/--'
    filename = os.listdir(path)[0]
    df = pd.read_excel(path+'/'+filename)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x90 in position 10: invalid start byte

In [134]:
path = 'ecommercedb_data/'+'germany'.title()
filename1 = [x for x in os.listdir(path) if '2106' in x][0]
logger.info('filename 1 is {}'.format(filename1))
files = os.listdir(path+ '/'+filename1)
logger.info('files are {}'.format(files))
file_vals = []
logger.info('fuzzy matching with {}'.format('Spirits'))
for file in files:
    file_vals.append(fuzz.ratio('Spirits',file))
logger.info('fuzzy matching with {}'.format(file_vals))
filename2 = files[np.argmax(file_vals)]
logger.info('filename 2 is {}'.format(filename2))
path = path+'/'+filename1+'/'+filename2
req_file = os.listdir(path)[0]
logger.info('required file path is {}'.format(path+'/'+req_file))
df = pd.read_csv(path+'/'+req_file)
df.fillna('NULL',inplace = True)

2023-02-10 13:52:55.171 | INFO     | __main__:<module>:3 - filename 1 is 2106_Food-Personal-Care
2023-02-10 13:52:55.172 | INFO     | __main__:<module>:5 - files are ['--', '2202_Food-Beverages', '3304_Personal-Care']
2023-02-10 13:52:55.176 | INFO     | __main__:<module>:7 - fuzzy matching with Spirits
2023-02-10 13:52:55.182 | INFO     | __main__:<module>:10 - fuzzy matching with [0, 15, 16]
2023-02-10 13:52:55.183 | INFO     | __main__:<module>:12 - filename 2 is 3304_Personal-Care
2023-02-10 13:52:55.186 | INFO     | __main__:<module>:15 - required file path is ecommercedb_data/Germany/2106_Food-Personal-Care/3304_Personal-Care/ecommercedb_Germany_Food-Personal-Care_Personal-Care_combined.xlsx


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x95 in position 10: invalid start byte

In [122]:

descp[descp['HSN']==int('0701')]['DESCP'].tolist()[0]

'0701 POTATOES, FRESH OR CHILLED Unit'

In [119]:
descp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   34 non-null     int64 
 1   HSN     34 non-null     int64 
 2   DESCP   34 non-null     object
dtypes: int64(2), object(1)
memory usage: 944.0+ bytes
